# AB Test Case Study 
Below I have been tasked with figuring out whether star ratings or guest ratings on a booking page are more impactful for driving purchases. I have been given a simulated dataset as well as an uplift calculator on excel. Below I am going to leverage my insights with python to study that data more closely and gain insights. 

In [1]:
import pandas as pd
from scipy.stats import norm
from sklearn.metrics import r2_score
import numpy as np
from scipy import stats
from scipy.stats import linregress
import scipy

In [2]:
bookings = pd.read_csv('/Users/aqureshi/Desktop/DS 021720/expedia_casestudy/HCOM_Product_Analytics_AB_Test_Case_Study/AB_Test_Case_Study_Dataset.csv')
bookings.head()

,local_date,num_unique_viewers,new_visitor_ind,site_sectn_name,platform_type,experiment_test_id,variant_code,brwsr_typ_name,property_parnt_chain_name,property_super_regn_name,length_of_stay,posa_country,num_unique_purchasers,total_booking_value_USD,net_bkg_room_nights
0,01/04/2018,1,return,BOOKING ENTER DETAILS,Desktop,MVT1225,H1225:002.006,Google,Motel One,EMEA,Short Break less than 4,SWEDEN,1,232.062456,2
1,01/04/2018,1,return,BOOKING ENTER DETAILS,Desktop,MVT1225,H1225:002.000,Google,Independent,APAC,Short Break less than 4,HONG KONG,0,0.000000,0
2,01/04/2018,1,new,BOOKING ENTER DETAILS,Mob :: TabWeb,MVT1225,H1225:002.000,Unknown,Independent,AMER,Long Break more than 5,United States of America,0,0.000000,0
3,01/04/2018,1,return,BOOKING ENTER DETAILS,Desktop,MVT1225,H1225:002.000,Google,Vagabond Inn,AMER,Long Break more than 5,United States of America,0,0.000000,0
4,01/04/2018,1,return,BOOKING ENTER DETAILS,Desktop,MVT1225,H1225:002.006,Unknown,Inter Hotel,EMEA,Short Break less than 4,FRANCE,1,87.437996,1


Below I would like to take a look at the different regions. This is important for testing and studying the results of how well each region has done with generating profits

In [3]:
bookings['property_super_regn_name'].value_counts()

AMER          167868
EMEA           94436
APAC           35578
LATAM          21670
Unknown         1670
Not Mapped        20
Name: property_super_regn_name, dtype: int64

In [4]:
unique_purchases_global = bookings.groupby(['variant_code', 'num_unique_purchasers']).count()
unique_purchases_global

local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                           92948               92948   
              1                           67641               67641   
              2                              30                  30   
              3                               1                   1   
              4                               1                   1   
H1225:002.006 0                           92411               92411   
              1                           68178               68178   
              2                              30                  30   
              3                               1                   1   
              4                               1                   1   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                92948            92948   
              1                                67641            67641   
              2                                   30               30   
              3                                    1                1   
              4                                    1                1   
H1225:002.006 0                                92411            92411   
              1                                68178            68178   
              2                                   30               30   
              3                                    1                1   
              4                                    1                1   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                              92948               92948   
              1                              67641               67641   
              2                                 30                  30   
              3                                  1                   1   
              4                                  1                   1   
H1225:002.006 0                              92411               92411   
              1                              68178               68178   
              2                                 30                  30   
              3                                  1                   1   
              4                                  1                   1   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                               92948   
              1                               67641   
              2                                  30   
              3                                   1   
              4                                   1   
H1225:002.006 0                               92411   
              1                               68178   
              2                                  30   
              3                                   1   
              4                                   1   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                          92948   
              1                                          67641   
              2                                             30   
              3                                              1   
              4                                              1   
H1225:002.006 0                                          92411   
              1                                          68178   
              2                                             30   
              3                                              1   
              4                

# Global Results for test vs control groups
After splitting the variant codes, I then decided to count the amount of unique purchases. As you can see, anything greater than 0 is considered as a booking. Since we are comparing booked vs not booked as a determining factor we can compare 0 vs 1,2,3,4 and sum together the latter values. 

In [5]:
bookings['num_unique_purchasers'] = bookings['num_unique_purchasers'].apply(lambda x: 1 if x > 0 else 0)

## I am going to create a dataframe that best represents the data shown in the calculator to keep the data analysis that I conduct consistent and ledgable

I am going to start by getting the total amount of booking value in USD. This will help to give me an overall view and comparison on how well the original AB test of star rating vs guest ratings as well as any other tests I conduct.Lastly for every test I conduct I will obtain the r**2 to examine the accuracy of each test. 

In [6]:
unique_purchases_global = bookings.groupby(['variant_code', 'num_unique_purchasers']).count()
money_generated = bookings['total_booking_value_USD'].sum()
money_generated_percentage = "{:.2%}".format(money_generated/money_generated)
print(money_generated_percentage)
unique_purchases_global

100.00%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                           92948               92948   
              1                           67673               67673   
H1225:002.006 0                           92411               92411   
              1                           68210               68210   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                92948            92948   
              1                                67673            67673   
H1225:002.006 0                                92411            92411   
              1                                68210            68210   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                              92948               92948   
              1                              67673               67673   
H1225:002.006 0                              92411               92411   
              1                              68210               68210   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                               92948   
              1                               67673   
H1225:002.006 0                               92411   
              1                               68210   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                          92948   
              1                                          67673   
H1225:002.006 0                                          92411   
              1                                          68210   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                         92948           92948   
              1                                         67673           67673   
H1225:002.006 0                                         92411           92411   
              1                                         68210           68210   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                             92948                    92948   
              1                             67673                    67673   
H1225:002.006 0                             92411                    92411   
              1                             68210                    68210   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                    92948  
              1                                    67673  
H1225:002.006 0                                    92411  
              1                                    68210

The next series of tests that I will look at are a part of my exploratory data analysis. This is important because I'd like to see if there are certain attributes that may have more of an impact with guest ratings rather than star ratings. If guest ratings are to be shown these tests will help to conclude whether guest ratings should be applied across the entire website or if Expedia should target specific attributes.

I am going to start by looking at whether `New Viewerss` make an impact, if so, how much of an impact compared to `Return Viewers`

In [7]:
#New customer groupby
filtered_new_customer = bookings.query('new_visitor_ind=="new"').groupby(['variant_code', 'num_unique_purchasers']).count()
filter_new = bookings.query('new_visitor_ind=="new"')['total_booking_value_USD'].sum() 
new_cust_percentage = "{:.2%}".format(filter_new/money_generated)
print(new_cust_percentage)
filtered_new_customer

23.47%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                           44030               44030   
              1                           23459               23459   
H1225:002.006 0                           43873               43873   
              1                           23616               23616   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                44030            44030   
              1                                23459            23459   
H1225:002.006 0                                43873            43873   
              1                                23616            23616   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                              44030               44030   
              1                              23459               23459   
H1225:002.006 0                              43873               43873   
              1                              23616               23616   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                               44030   
              1                               23459   
H1225:002.006 0                               43873   
              1                               23616   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                          44030   
              1                                          23459   
H1225:002.006 0                                          43873   
              1                                          23616   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                         44030           44030   
              1                                         23459           23459   
H1225:002.006 0                                         43873           43873   
              1                                         23616           23616   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                             44030                    44030   
              1                             23459                    23459   
H1225:002.006 0                             43873                    43873   
              1                             23616                    23616   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                    44030  
              1                                    23459  
H1225:002.006 0                                    43873  
              1                                    23616

In [8]:
#Return customer groupby
filtered_return_customer = bookings.query('new_visitor_ind=="return"').groupby(['variant_code', 'num_unique_purchasers']).count()
filter_return = bookings.query('new_visitor_ind=="return"')['total_booking_value_USD'].sum() 
return_cust_percentage = "{:.2%}".format(filter_return/money_generated)
print(return_cust_percentage)
filtered_return_customer

76.53%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                           48918               48918   
              1                           44214               44214   
H1225:002.006 0                           48538               48538   
              1                           44594               44594   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                48918            48918   
              1                                44214            44214   
H1225:002.006 0                                48538            48538   
              1                                44594            44594   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                              48918               48918   
              1                              44214               44214   
H1225:002.006 0                              48538               48538   
              1                              44594               44594   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                               48918   
              1                               44214   
H1225:002.006 0                               48538   
              1                               44594   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                          48918   
              1                                          44214   
H1225:002.006 0                                          48538   
              1                                          44594   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                         48918           48918   
              1                                         44214           44214   
H1225:002.006 0                                         48538           48538   
              1                                         44594           44594   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                             48918                    48918   
              1                             44214                    44214   
H1225:002.006 0                             48538                    48538   
              1                             44594                    44594   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                    48918  
              1                                    44214  
H1225:002.006 0                                    48538  
              1                                    44594

It seems that `Return Viewers` make up a larger portion of sales overall. I want to now split up each region to identify whether specific areas of the world have more of the market share and how much of the sales make up that particular area. 

In [9]:
#EMEA groupby
emea = bookings.query('property_super_regn_name=="EMEA"').groupby(['variant_code', 'num_unique_purchasers']).count()
emea_money_generated = bookings.query('property_super_regn_name =="EMEA"')['total_booking_value_USD'].sum() 
emea_percentage = "{:.2%}".format(emea_money_generated/money_generated)
print(emea_percentage)
emea

33.63%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                           26165               26165   
              1                           21053               21053   
H1225:002.006 0                           25983               25983   
              1                           21235               21235   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                26165            26165   
              1                                21053            21053   
H1225:002.006 0                                25983            25983   
              1                                21235            21235   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                              26165               26165   
              1                              21053               21053   
H1225:002.006 0                              25983               25983   
              1                              21235               21235   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                               26165   
              1                               21053   
H1225:002.006 0                               25983   
              1                               21235   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                          26165   
              1                                          21053   
H1225:002.006 0                                          25983   
              1                                          21235   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                         26165           26165   
              1                                         21053           21053   
H1225:002.006 0                                         25983           25983   
              1                                         21235           21235   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                             26165                    26165   
              1                             21053                    21053   
H1225:002.006 0                             25983                    25983   
              1                             21235                    21235   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                    26165  
              1                                    21053  
H1225:002.006 0                                    25983  
              1                                    21235

In [10]:
#AMER groupby
amer = bookings.query('property_super_regn_name=="AMER"').groupby(['variant_code', 'num_unique_purchasers']).count()
amer_money_generated = bookings.query('property_super_regn_name =="AMER"')['total_booking_value_USD'].sum()
amer_percentage = "{:.2%}".format(amer_money_generated/money_generated)
print(amer_percentage)
amer

53.86%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                           45709               45709   
              1                           38225               38225   
H1225:002.006 0                           45414               45414   
              1                           38520               38520   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                45709            45709   
              1                                38225            38225   
H1225:002.006 0                                45414            45414   
              1                                38520            38520   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                              45709               45709   
              1                              38225               38225   
H1225:002.006 0                              45414               45414   
              1                              38520               38520   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                               45709   
              1                               38225   
H1225:002.006 0                               45414   
              1                               38520   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                          45709   
              1                                          38225   
H1225:002.006 0                                          45414   
              1                                          38520   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                         45709           45709   
              1                                         38225           38225   
H1225:002.006 0                                         45414           45414   
              1                                         38520           38520   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                             45709                    45709   
              1                             38225                    38225   
H1225:002.006 0                             45414                    45414   
              1                             38520                    38520   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                    45709  
              1                                    38225  
H1225:002.006 0                                    45414  
              1                                    38520

In [11]:
#LATAM groupby
latam = bookings.query('property_super_regn_name=="LATAM"').groupby(['variant_code', 'num_unique_purchasers']).count()
latam_money_generated = bookings.query('property_super_regn_name =="LATAM"')['total_booking_value_USD'].sum()
latam_percentage = "{:.2%}".format(latam_money_generated/money_generated)
print(latam_percentage)
latam

3.98%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                            8552                8552   
              1                            2283                2283   
H1225:002.006 0                            8527                8527   
              1                            2308                2308   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                 8552             8552   
              1                                 2283             2283   
H1225:002.006 0                                 8527             8527   
              1                                 2308             2308   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                               8552                8552   
              1                               2283                2283   
H1225:002.006 0                               8527                8527   
              1                               2308                2308   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                                8552   
              1                                2283   
H1225:002.006 0                                8527   
              1                                2308   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                           8552   
              1                                           2283   
H1225:002.006 0                                           8527   
              1                                           2308   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                          8552            8552   
              1                                          2283            2283   
H1225:002.006 0                                          8527            8527   
              1                                          2308            2308   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                              8552                     8552   
              1                              2283                     2283   
H1225:002.006 0                              8527                     8527   
              1                              2308                     2308   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                     8552  
              1                                     2283  
H1225:002.006 0                                     8527  
              1                                     2308

In [12]:
#APAC groupby
apac = bookings.query('property_super_regn_name=="APAC"').groupby(['variant_code', 'num_unique_purchasers']).count()
apac_money_generated = bookings.query('property_super_regn_name =="APAC"')['total_booking_value_USD'].sum()
apac_percentage = "{:.2%}".format(apac_money_generated/money_generated)
print(apac_percentage)
apac

8.09%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                           11971               11971   
              1                            5818                5818   
H1225:002.006 0                           11938               11938   
              1                            5851                5851   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                11971            11971   
              1                                 5818             5818   
H1225:002.006 0                                11938            11938   
              1                                 5851             5851   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                              11971               11971   
              1                               5818                5818   
H1225:002.006 0                              11938               11938   
              1                               5851                5851   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                               11971   
              1                                5818   
H1225:002.006 0                               11938   
              1                                5851   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                          11971   
              1                                           5818   
H1225:002.006 0                                          11938   
              1                                           5851   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                         11971           11971   
              1                                          5818            5818   
H1225:002.006 0                                         11938           11938   
              1                                          5851            5851   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                             11971                    11971   
              1                              5818                     5818   
H1225:002.006 0                             11938                    11938   
              1                              5851                     5851   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                    11971  
              1                                     5818  
H1225:002.006 0                                    11938  
              1                                     5851

In [13]:
#unknown groupby
unknown = bookings.query('property_super_regn_name=="Unknown"').groupby(['variant_code', 'num_unique_purchasers']).count()
unknown_money_generated = bookings.query('property_super_regn_name =="Unknown"')['total_booking_value_USD'].sum()
unknown_percentage = "{:.2%}".format(unknown_money_generated/money_generated)
print(unknown_percentage)
unknown

0.44%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                             543                 543   
              1                             292                 292   
H1225:002.006 0                             541                 541   
              1                             294                 294   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                  543              543   
              1                                  292              292   
H1225:002.006 0                                  541              541   
              1                                  294              294   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                                543                 543   
              1                                292                 292   
H1225:002.006 0                                541                 541   
              1                                294                 294   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                                 543   
              1                                 292   
H1225:002.006 0                                 541   
              1                                 294   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                            543   
              1                                            292   
H1225:002.006 0                                            541   
              1                                            294   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                           543             543   
              1                                           292             292   
H1225:002.006 0                                           541             541   
              1                                           294             294   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                               543                      543   
              1                               292                      292   
H1225:002.006 0                               541                      541   
              1                               294                      294   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                      543  
              1                                      292  
H1225:002.006 0                                      541  
              1                                      294

In [14]:
#not mapped
#unknown groupby
not_mapped = bookings.query('property_super_regn_name=="Not Mapped"').groupby(['variant_code', 'num_unique_purchasers']).count()
not_mapped_money_generated = bookings.query('property_super_regn_name =="Not Mapped"')['total_booking_value_USD'].sum()
not_mapped_percentage = "{:.2%}".format(not_mapped_money_generated/money_generated)
print(not_mapped_percentage)
not_mapped

0.00%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                               8                   8   
              1                               2                   2   
H1225:002.006 0                               8                   8   
              1                               2                   2   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                    8                8   
              1                                    2                2   
H1225:002.006 0                                    8                8   
              1                                    2                2   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                                  8                   8   
              1                                  2                   2   
H1225:002.006 0                                  8                   8   
              1                                  2                   2   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                                   8   
              1                                   2   
H1225:002.006 0                                   8   
              1                                   2   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                              8   
              1                                              2   
H1225:002.006 0                                              8   
              1                                              2   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                             8               8   
              1                                             2               2   
H1225:002.006 0                                             8               8   
              1                                             2               2   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                                 8                        8   
              1                                 2                        2   
H1225:002.006 0                                 8                        8   
              1                                 2                        2   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                        8  
              1                                        2  
H1225:002.006 0                                        8  
              1                                        2

I now would like to see the difference between short and long stay and if there is a difference in money generated, what is the percentage of each of them. 

In [15]:
#Short Break groupby
short_break = bookings.query('length_of_stay=="Short Break less than 4"').groupby(['variant_code', 'num_unique_purchasers']).count()
short_break_generated = bookings.query('length_of_stay =="Short Break less than 4"')['total_booking_value_USD'].sum()
short_break_percentage = "{:.2%}".format(short_break_generated/money_generated)
print(short_break_percentage)
short_break

29.50%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                           27701               27701   
              1                           20276               20276   
H1225:002.006 0                           27618               27618   
              1                           20501               20501   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                27701            27701   
              1                                20276            20276   
H1225:002.006 0                                27618            27618   
              1                                20501            20501   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                              27701               27701   
              1                              20276               20276   
H1225:002.006 0                              27618               27618   
              1                              20501               20501   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                               27701   
              1                               20276   
H1225:002.006 0                               27618   
              1                               20501   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                          27701   
              1                                          20276   
H1225:002.006 0                                          27618   
              1                                          20501   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                         27701           27701   
              1                                         20276           20276   
H1225:002.006 0                                         27618           27618   
              1                                         20501           20501   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                             27701                    27701   
              1                             20276                    20276   
H1225:002.006 0                             27618                    27618   
              1                             20501                    20501   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                    27701  
              1                                    20276  
H1225:002.006 0                                    27618  
              1                                    20501

In [16]:
#Long Break groupby
long_break = bookings.query('length_of_stay=="Long Break more than 5"').groupby(['variant_code', 'num_unique_purchasers']).count()
long_break_generated = bookings.query('length_of_stay =="Long Break more than 5"')['total_booking_value_USD'].sum()
long_break_percentage = "{:.2%}".format(long_break_generated/money_generated)
print(long_break_percentage)
long_break

70.50%


local_date  num_unique_viewers  \
variant_code  num_unique_purchasers                                   
H1225:002.000 0                           65247               65247   
              1                           47397               47397   
H1225:002.006 0                           64793               64793   
              1                           47709               47709   

                                     new_visitor_ind  site_sectn_name  \
variant_code  num_unique_purchasers                                     
H1225:002.000 0                                65247            65247   
              1                                47397            47397   
H1225:002.006 0                                64793            64793   
              1                                47709            47709   

                                     platform_type  experiment_test_id  \
variant_code  num_unique_purchasers                                      
H1225:002.000 0                              65247               65247   
              1                              47397               47397   
H1225:002.006 0                              64793               64793   
              1                              47709               47709   

                                     brwsr_typ_name  \
variant_code  num_unique_purchasers                   
H1225:002.000 0                               65247   
              1                               47397   
H1225:002.006 0                               64793   
              1                               47709   

                                     property_parnt_chain_name  \
variant_code  num_unique_purchasers                              
H1225:002.000 0                                          65247   
              1                                          47397   
H1225:002.006 0                                          64793   
              1                                          47709   

                                     property_super_regn_name  length_of_stay  \
variant_code  num_unique_purchasers                                             
H1225:002.000 0                                         65247           65247   
              1                                         47397           47397   
H1225:002.006 0                                         64793           64793   
              1                                         47709           47709   

                                     posa_country  total_booking_value_USD  \
variant_code  num_unique_purchasers                                          
H1225:002.000 0                             65247                    65247   
              1                             47397                    47397   
H1225:002.006 0                             64793                    64793   
              1                             47709                    47709   

                                     net_bkg_room_nights  
variant_code  num_unique_purchasers                       
H1225:002.000 0                                    65247  
              1                                    47397  
H1225:002.006 0                                    64793  
              1                                    47709

It seems that longer breaks are what most customers look at. <br>
I would like to now format all of the tests that I have done and put it into a data format similar to the `uplift` calculator. I will start by formating the tests conducted properly and then run it through a `for loop` so that it will make things less tedious. I have chosen to put all of the tests next to one another for comparison. 

In [17]:
cleaned_data = [
    {'Test Name': 'Global Test', 
     '% Turnover': money_generated_percentage,
     'Control Unique Visitors':unique_purchases_global.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':unique_purchases_global.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':unique_purchases_global.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':unique_purchases_global.iloc[3]['num_unique_viewers']
    },
    {'Test Name': 'New Customer Test', 
     '% Turnover': new_cust_percentage,
     'Control Unique Visitors':filtered_new_customer.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':filtered_new_customer.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':filtered_new_customer.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':filtered_new_customer.iloc[3]['num_unique_viewers']
    },
    {'Test Name': 'Return Customer Test',
     '% Turnover': return_cust_percentage,
     'Control Unique Visitors':filtered_return_customer.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':filtered_return_customer.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':filtered_return_customer.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':filtered_return_customer.iloc[3]['num_unique_viewers']
    },
    {'Test Name': 'EMEA',
     '% Turnover': emea_percentage,
     'Control Unique Visitors':emea.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':emea.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':emea.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':emea.iloc[3]['num_unique_viewers']    
    }, 
    {'Test Name': 'America',
     '% Turnover': amer_percentage,
     'Control Unique Visitors':amer.iloc[0]['num_unique_viewers'], 
     'Control Unique Purchasers':amer.iloc[1]['num_unique_viewers'], 
     'Challenger Unique Visitors':amer.iloc[2]['num_unique_viewers'], 
     'Challenger Converted Customers':amer.iloc[3]['num_unique_viewers']
    }, 
    {'Test Name': 'Latin America',
     '% Turnover': latam_percentage,
     'Control Unique Visitors':latam.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':latam.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':latam.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':latam.iloc[3]['num_unique_viewers']
    },
    {'Test Name': 'APAC',
     '% Turnover': apac_percentage,
     'Control Unique Visitors':apac.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':apac.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':apac.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':apac.iloc[3]['num_unique_viewers']
    },
    {'Test Name': 'Unknown',
     '% Turnover': unknown_percentage,
     'Control Unique Visitors':unknown.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':unknown.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':unknown.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':unknown.iloc[3]['num_unique_viewers']
    },
    {'Test Name': 'Not Mapped',
     '% Turnover': not_mapped_percentage,
     'Control Unique Visitors':not_mapped.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':not_mapped.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':not_mapped.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':not_mapped.iloc[3]['num_unique_viewers']
    },  
    {'Test Name': 'Short break',
     '% Turnover': short_break_percentage,
     'Control Unique Visitors':short_break.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':short_break.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':short_break.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':short_break.iloc[3]['num_unique_viewers']
    },
    {'Test Name': 'Long break',
     '% Turnover': long_break_percentage,
     'Control Unique Visitors':long_break.iloc[0]['num_unique_viewers'],
     'Control Unique Purchasers':long_break.iloc[1]['num_unique_viewers'],
     'Challenger Unique Visitors':long_break.iloc[2]['num_unique_viewers'],
     'Challenger Converted Customers':long_break.iloc[3]['num_unique_viewers']
    },
]

In [18]:
confidence_interval = abs(norm.ppf((1 - 0.90) / 2))
results = pd.DataFrame(columns=(
    'Test Name',
    '% Turnover',
    'Total Orders',
    'Control Unique Visitors',
    'Control Unique Purchasers',
    'Control CVR',
    'Challenger Unique Visitors',
    'Challenger Converted Customers',
    'Challenger CVR',
    'Diff',
    'Sig',
    'Low',
    'Obs',
    'High'
))
for data in cleaned_data:
    control_unique_visitors = data['Control Unique Visitors']
    challenger_unique_visitors = data['Challenger Unique Visitors']
    control_CVR = data['Control Unique Purchasers'] / data['Control Unique Visitors']
    challenger_CVR = data['Challenger Converted Customers'] / data['Challenger Unique Visitors']
    diff = (challenger_CVR - control_CVR) / control_CVR
    z_score = (challenger_CVR - control_CVR) / \
    ( \
        (challenger_CVR * (1 - challenger_CVR) / challenger_unique_visitors) + \
        (control_CVR * (1 - control_CVR) / control_unique_visitors) \
    )**0.5
    delta_CVR = challenger_CVR - control_CVR
    standard_dev = ( \
        (challenger_CVR * (1 - challenger_CVR) / challenger_unique_visitors) + \
        (control_CVR * (1 - control_CVR) / control_unique_visitors) \
    )**0.5
    results = results.append([{
        'Test Name':data['Test Name'], 
        '% Turnover': data['% Turnover'],
        'Total Orders': data['Control Unique Visitors'] + data['Control Unique Purchasers'],
        'Control Unique Visitors': control_unique_visitors,
        'Control Unique Purchasers': data['Control Unique Purchasers'],
        'Control CVR': "{:.2%}".format(control_CVR),
        'Challenger Unique Visitors': challenger_unique_visitors,
        'Challenger Converted Customers': data['Challenger Converted Customers'],
        'Challenger CVR' : "{:.2%}".format(challenger_CVR),
        'Diff': "{:.2%}".format(diff),
        'Sig': "{:.0%}".format(norm.cdf(abs(z_score))),
        'Low': "{:.2%}".format((delta_CVR - confidence_interval * standard_dev) / control_CVR),
        'Obs': "{:.2%}".format(diff),
        'High': "{:.2%}".format((delta_CVR + confidence_interval * standard_dev) / control_CVR),
    }], ignore_index=True)
results

,Test Name,% Turnover,Total Orders,Control Unique Visitors,Control Unique Purchasers,Control CVR,Challenger Unique Visitors,Challenger Converted Customers,Challenger CVR,Diff,Sig,Low,Obs,High
0,Global Test,100.00%,160621,92948,67673,72.81%,92411,68210,73.81%,1.38%,100%,0.92%,1.38%,1.84%
1,New Customer Test,23.47%,67489,44030,23459,53.28%,43873,23616,53.83%,1.03%,95%,-0.01%,1.03%,2.07%
2,Return Customer Test,76.53%,93132,48918,44214,90.38%,48538,44594,91.87%,1.65%,100%,1.32%,1.65%,1.98%
3,EMEA,33.63%,47218,26165,21053,80.46%,25983,21235,81.73%,1.57%,100%,0.87%,1.57%,2.27%
4,America,53.86%,83934,45709,38225,83.63%,45414,38520,84.82%,1.43%,100%,0.95%,1.43%,1.90%
5,Latin America,3.98%,10835,8552,2283,26.70%,8527,2308,27.07%,1.39%,71%,-2.79%,1.39%,5.57%
6,APAC,8.09%,17789,11971,5818,48.60%,11938,5851,49.01%,0.85%,74%,-1.34%,0.85%,3.03%
7,Unknown,0.44%,835,543,292,53.78%,541,294,54.34%,1.06%,57%,-8.20%,1.06%,10.32%
8,Not Mapped,0.00%,10,8,2,25.00%,8,2,25.00%,0.00%,50%,-142.45%,0.00%,142.45%
9,Short break,29.50%,47977,27701,20276,73.20%,27618,20501,74.23%,1.41%,100%,0.57%,1.41%,2.25%


# Conclusion:
From the above, I can conclude that changing the star ratings to guest ratings would make a positive impact on the overall booking website. <br>
<br>
I have taken it upon myself to conduct other tests that look closer into which columns and subgroups make more of an impact(Diff) with guest ratings. I can conclude that all of them have had a positive impact with guest ratings. I 
The most obvious are that a majority of people who do make bookings are those that are return customers, the next big difference that I see is that EMEA region tends to get booked most frequently, followed by America, as well at Latin American followed by short breaks, long breaks and the global test that. <br>
<br>
Note: `Unknown` and `Not Mapped` hae high conversion rates, but its important to keep in mind that they do not really have a lot of vistors or purchases. I would consider these to be outliers. 